In [1]:
import numpy as np
import pickle

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense,Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout , GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
import os

In [2]:
files = os.listdir("./DATA")
x = []
x_load = []
y = []
y_load = []

In [3]:
def load_data():
    count = 0
    for file in files:
        file = "./DATA/" + file
        x = np.load(file, encoding = "latin1", allow_pickle=True)
        x = x.astype('float32') / 255.
        x = x[0:10000, :]
        x_load.append(x)
        y = [count for _ in range(10000)]
        count += 1
        y = np.array(y).astype('float32')
        y = y.reshape(y.shape[0], 1)
        y_load.append(y)
    return x_load, y_load, count


In [4]:
features, labels,DataCount = load_data()
features = np.array(features).astype('float32')

labels = np.array(labels).astype('float32')
features=features.reshape(features.shape[0]*features.shape[1],features.shape[2])

labels=labels.reshape(labels.shape[0]*labels.shape[1],labels.shape[2])

with open("PickleData/features", "wb") as f:
    pickle.dump(features, f, protocol=4)
with open("PickleData/labels", "wb") as f:
    pickle.dump(labels, f, protocol=4)

print(DataCount)

10


In [5]:
def keras_model(image_x, image_y):
    num_of_classes = DataCount
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list

In [6]:
def mobileNet(image_x, image_y): 
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNet
    num_of_classes = DataCount

    MobileNet = MobileNet(weights = None, include_top = False, input_shape=(image_x, image_y,1))
    for layer in MobileNet.layers:
        layer.trainable = True

    input_model = MobileNet.output
    
    input_model = GlobalAveragePooling2D()(input_model)
    input_model = Dense(512, activation = 'relu')(input_model)
    input_model = Dense(128, activation = 'relu')(input_model)
    output_model = Dense(num_of_classes, activation = 'softmax')(input_model)

    model = Model(inputs = MobileNet.input , outputs = output_model)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "QuickDraw_Mobile.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list 

In [7]:
Picklepath = 'PickleData/'
def loadFromPickle():
    with open(Picklepath+"features", "rb") as f:
        features = np.array(pickle.load(f))
    with open(Picklepath+"labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels

def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

def prepress_labels(labels):
    labels = utils.to_categorical(labels)
    return labels

In [8]:
ModelPath = "Model/"

def main(modelName):
    if not os.path.exists(ModelPath):
        os.makedirs(ModelPath)
    """modelName : cnn / mobilenet"""

    features, labels = loadFromPickle()
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,
                                                        test_size=0.1)
    if modelName == 'cnn':
        train_x = train_x.reshape(train_x.shape[0], 28, 28, 1) 
        test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
        # plt.imshow(train_x[0], cmap="Greys")
        # plt.show()
        model, callbacks_list = keras_model(28,28)
        model.summary()
        model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64,
                callbacks=[TensorBoard(log_dir="QuickDraw")])
        model.save(ModelPath+'QuickDraw.h5')
    elif modelName == 'mobilenet': 
        train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
        test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
        # becuase MobileNet InputSize at least (32,32)
        paddings = tf.constant([[0,0],[2,2],[2,2],[0,0]])
        train_x = tf.pad(train_x, paddings , mode = 'constant' , constant_values = 0)
        test_x = tf.pad(test_x, paddings, mode = 'constant', constant_values = 0)
        # plt.imshow(train_x[0], cmap="Greys")
        # plt.show()
        model, callbacks_list = mobileNet(32,32)
        model.summary()
        model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64,
                callbacks=[TensorBoard(log_dir="QuickDraw_mobile")])
        model.save(ModelPath+'QuickDraw_mobileNet.h5')

In [9]:
main('mobilenet') 
# main('cnn')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 33, 33, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32)        128   